In [2]:
"""
Automated Exploratory Data Analysis (EDA) Script
---------------------------------------------

This script performs a comprehensive exploratory data analysis on an Excel dataset
and writes the results to a text file. It's designed to provide a quick but thorough
overview of the data's characteristics.

Key Features:
- Loads Excel data and performs basic validation
- Generates comprehensive EDA including:
  * Basic dataset information and sample rows
  * Numerical column statistics
  * Missing value analysis
  * Text column exploration
  * Value counts for categorical columns
  * Text length analysis for description fields
  * Duplicate row detection
  * Correlation analysis for numerical columns

Output:
- All analysis results are written to 'eda_output.txt'
- Progress indicators and error messages are displayed during execution

Usage:
- Ensure 'data_18_mos.xlsx' exists in the same directory
- Run the script to generate the EDA report
- Results will be saved in 'eda_output.txt'

Dependencies:
- pandas
- numpy
"""

# Import necessary libraries
import pandas as pd
import numpy as np
from contextlib import redirect_stdout
import io

# Define the output file path
output_file = 'eda_output.txt'

def configure_pandas_display() -> None:
    """
    Configure pandas to show all columns and full column width in outputs.
    Limit max rows displayed to prevent excessive output.
    """
    pd.set_option('display.max_rows', 15)  # Limit max rows displayed
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 1000)

def load_data(file_path: str) -> pd.DataFrame:
    """
    Load data from the given Excel file into a pandas DataFrame.
    """
    try:
        df = pd.read_excel(file_path)
        print(f"✅ Successfully loaded data from '{file_path}'.")
        return df
    except FileNotFoundError:
        print(f"❌ Error: The file '{file_path}' was not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ An error occurred while loading the file: {e}")
        return pd.DataFrame()

def print_basic_info(df: pd.DataFrame) -> None:
    """
    Print basic information about the DataFrame.
    """
    print("\n=== BASIC INFO ===")
    print("\n--- First 10 Rows ---")
    print(df.head(10).to_string(index=False))

    print("\n--- DataFrame Info ---")
    buffer = io.StringIO()
    df.info(verbose=True, buf=buffer)
    info_str = buffer.getvalue()
    print(info_str)

    print("\n--- Memory Usage ---")
    memory_usage = df.memory_usage(deep=True).sort_values(ascending=False)
    print(memory_usage.to_string())

    print(f"\n--- Shape of the DataFrame: {df.shape[0]} rows, {df.shape[1]} columns ---")

def print_dataframe_sample(df: pd.DataFrame, sample_size: int = 15) -> None:
    """
    Print a random sample of the DataFrame and the last few rows.
    """
    print("\n=== DATAFRAME SAMPLE ===")
    print(f"\n--- Random Sample of {sample_size} Rows ---")
    sample_df = df.sample(n=sample_size, random_state=42)  # Fixed seed for reproducibility
    print(sample_df.to_string(index=False))

    print("\n--- Last 10 Rows ---")
    print(df.tail(10).to_string(index=False))

def describe_numerical_columns(df: pd.DataFrame) -> None:
    """
    Describe numerical columns in the DataFrame.
    """
    print("\n=== DESCRIBE NUMERICAL COLUMNS ===")
    numerical_cols = df.select_dtypes(include=[np.number])
    if not numerical_cols.empty:
        print(numerical_cols.describe().to_string())
    else:
        print("ℹ️ No numerical columns in the dataset. Skipping describe.")

def analyze_missing_values(df: pd.DataFrame) -> None:
    """
    Analyze and print missing values summary.
    """
    print("\n=== MISSING VALUES SUMMARY ===")
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Values': missing_values,
        'Percent': missing_percentage
    }).sort_values('Missing Values', ascending=False)
    print(missing_summary.to_string())

def explore_text_columns(df: pd.DataFrame) -> None:
    """
    Explore text columns and print relevant information.
    """
    print("\n=== TEXT COLUMN EXPLORATION ===")
    text_columns = ['Description', 'Short Description']  # Add other text columns as needed
    for text_col in text_columns:
        if text_col in df.columns:
            null_count = df[text_col].isnull().sum()
            print(f"\n--- Column '{text_col}' ---")
            print(f"Number of null entries: {null_count}")
            unique_count = df[text_col].nunique()
            print(f"Unique entries: {unique_count}")
        else:
            print(f"\nℹ️ Column '{text_col}' not found. Skipping related checks.")

def analyze_value_counts(df: pd.DataFrame) -> None:
    """
    Analyze value counts for specific columns.
    """
    print("\n=== VALUE COUNTS ===")
    target_columns = ['Category', 'Subcategory', 'Assignment Group']
    for col in target_columns:
        if col in df.columns:
            print(f"\n--- Value Counts for '{col}' (Top 20) ---")
            value_counts = df[col].value_counts(dropna=False).head(20)
            print(value_counts.to_string())
        else:
            print(f"\nℹ️ Column '{col}' not found. Skipping value counts.")

def analyze_text_lengths(df: pd.DataFrame) -> None:
    """
    Analyze text lengths for specific columns and provide summary statistics.
    """
    print("\n=== TEXT LENGTH ANALYSIS ===")
    text_columns = ['Short Description', 'Description']
    for text_col in text_columns:
        if text_col in df.columns:
            new_col_name = text_col.lower().replace(' ', '_') + "_length"
            df[new_col_name] = df[text_col].astype(str).str.len()
            print(f"\n--- '{text_col}' Length Statistics ---")
            length_stats = df[new_col_name].describe()
            print(length_stats.to_string())
        else:
            print(f"\nℹ️ Column '{text_col}' not found. Skipping text length analysis.")

def analyze_duplicates(df: pd.DataFrame) -> None:
    """
    Analyze and print duplicate rows summary.
    """
    print("\n=== DUPLICATE ROW ANALYSIS ===")
    duplicate_count = df.duplicated().sum()
    print(f"🔍 Total duplicate rows: {duplicate_count}")
    if duplicate_count > 0:
        print("ℹ️ Duplicate rows exist in the dataset.")
    else:
        print("✅ No duplicate rows found.")

def analyze_correlations(df: pd.DataFrame) -> None:
    """
    Analyze and print correlation matrix for numerical columns.
    """
    print("\n=== CORRELATION MATRIX ===")
    num_cols = df.select_dtypes(include=[np.number])
    if not num_cols.empty:
        corr_matrix = num_cols.corr()
        print(corr_matrix.to_string())
    else:
        print("ℹ️ No numerical columns available for correlation analysis.")

def run_full_analysis(df: pd.DataFrame) -> None:
    """
    Execute all EDA functions in sequence.
    """
    print_basic_info(df)
    print_dataframe_sample(df, sample_size=15)  # Print a sample of 15 rows
    describe_numerical_columns(df)
    analyze_missing_values(df)
    explore_text_columns(df)
    analyze_value_counts(df)
    analyze_text_lengths(df)
    analyze_duplicates(df)
    analyze_correlations(df)

# Configure pandas display options
configure_pandas_display()

# Load the data
df = load_data('data_18_mos.xlsx')

# Check if DataFrame is not empty before proceeding
if not df.empty:
    # Redirect all print statements to the output file
    with open(output_file, 'w') as f:
        with redirect_stdout(f):
            run_full_analysis(df)

    print(f"✅ EDA complete. Results have been written to '{output_file}'.")
else:
    print("❌ DataFrame is empty. EDA aborted.")


✅ Successfully loaded data from 'data_18_mos.xlsx'.
✅ EDA complete. Results have been written to 'eda_output.txt'.
